# **Intercomparison Demo**

**Author:** Adam Yang (ayang115@umd.edu)

**Description** Demo for Intercomparison Script: extracts test points, provided from hand labeled maps, from industry covermaps (Harvest, Copernicus, ESA, GLAD) and determines accuracy across maps.


In [5]:
# Authenticate notebook for Earth Engine use. Only needs to be run once, but is required for functions to work!
!earthengine authenticate

Fetching credentials using gcloud

Successfully saved authorization token.


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=yK7gVWY7L0wtBc0Ha1LeNogydLI6ua&access_type=offline&code_challenge=jDUJGkqATKUbP5K1YhXJQSuokQ4eNkFaDCbmLb5hKAc&code_challenge_method=S256


Credentials saved to file: [C:\Users\adamf\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).


In [1]:
import sys
sys.path.append('../src')

In [2]:
from compare_covermaps import *
import pandas as pd

c:\Users\adamf\anaconda3\envs\intercomparison\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


**1. Extract testing points from CEO maps**

In [3]:
test_data = generate_test_data(TEST_PATHS)

Kenya
829
764
Tanzania_CEO_2019
2710
1537
Togo
355
352


In [4]:
test_data.head()

,lat,lon,class_probability,country,subset,geometry,binary
0,0.725122,34.386282,0.0,Kenya,testing,POINT (34.38628 0.72512),0
1,0.621939,34.466496,0.0,Kenya,testing,POINT (34.46650 0.62194),0
2,0.226497,34.054859,0.0,Kenya,testing,POINT (34.05486 0.22650),0
3,0.096834,33.951959,0.0,Kenya,testing,POINT (33.95196 0.09683),0
4,0.032072,33.968469,0.0,Kenya,testing,POINT (33.96847 0.03207),0


**2. Use testing data to extract points from EE maps**

In [5]:
harvest = extract_harvest(test_data)

sampling togo_cropland_binary
sampling kenya_cropland_binary
sampling Tanzania_cropland_2019


In [6]:
harvest.head()

,lat,lon,class_probability,country,subset,geometry,binary,harvest
0,0.725122,34.386282,0.0,Kenya,testing,POINT (34.38628 0.72512),0,1
1,0.621939,34.466496,0.0,Kenya,testing,POINT (34.46650 0.62194),0,1
2,0.226497,34.054859,0.0,Kenya,testing,POINT (34.05486 0.22650),0,0
3,0.096834,33.951959,0.0,Kenya,testing,POINT (33.95196 0.09683),0,0
4,0.032072,33.968469,0.0,Kenya,testing,POINT (33.96847 0.03207),0,0


In [7]:
covermaps = extract_covermaps(test_data)

sampling copernicus
sampling esa
sampling glad


In [8]:
covermaps.head()

,lat,lon,class_probability,country,subset,geometry,binary,cop,esa,glad
0,0.725122,34.386282,0.0,Kenya,testing,POINT (34.38628 0.72512),0,0,0,1
1,0.621939,34.466496,0.0,Kenya,testing,POINT (34.46650 0.62194),0,0,0,0
2,0.226497,34.054859,0.0,Kenya,testing,POINT (34.05486 0.22650),0,1,0,0
3,0.096834,33.951959,0.0,Kenya,testing,POINT (33.95196 0.09683),0,0,0,0
4,0.032072,33.968469,0.0,Kenya,testing,POINT (33.96847 0.03207),0,0,0,0


The 'binary' column contains recasted testing points that will be used as ground truth for the classification report. 

The 'binary' column contains recasted testing points that will be used as ground truth for the classification report. 

In [9]:
# Combine harvest and industry covermaps into one dataframe
covermaps["harvest"] = pd.merge(harvest, covermaps, on=["lat", "lon"], how="left")["harvest"]

**3. Evaluate maps**

In [11]:
covermap_results = []
for country, df in covermaps.groupby("country"):
    for dataset in ["harvest", "cop", "esa", "glad"]:
        dropped = df.dropna().copy()
        print(
            "dataset: " + dataset +
            " | country: " + country + 
            " | nan values dropped: " + str(len(df)-len(dropped)))
        
        covermap_results.append(generate_report(dataset, country, pred=dropped[dataset], true=dropped["binary"]))

covermap_results = pd.concat(covermap_results)

dataset: harvest | country: Kenya | nan values dropped: 0
dataset: cop | country: Kenya | nan values dropped: 0
dataset: esa | country: Kenya | nan values dropped: 0
dataset: glad | country: Kenya | nan values dropped: 0
dataset: harvest | country: Tanzania | nan values dropped: 0
dataset: cop | country: Tanzania | nan values dropped: 0
dataset: esa | country: Tanzania | nan values dropped: 0
dataset: glad | country: Tanzania | nan values dropped: 0
dataset: harvest | country: Togo | nan values dropped: 0
dataset: cop | country: Togo | nan values dropped: 0
dataset: esa | country: Togo | nan values dropped: 0
dataset: glad | country: Togo | nan values dropped: 0


In [13]:
covermap_results

,dataset,country,crop_f1,accuracy,crop_support,noncrop_support,crop_recall,noncrop_recall,crop_precision,noncrop_precision
0,harvest,Kenya,0.921514,0.888743,571,193,0.873905,0.932642,0.974609,0.714286
0,cop,Kenya,0.937824,0.905759,571,193,0.950963,0.772021,0.925043,0.841808
0,esa,Kenya,0.333333,0.397906,571,193,0.201401,0.979275,0.966387,0.293023
0,glad,Kenya,0.883978,0.835079,571,193,0.840630,0.818653,0.932039,0.634538
0,harvest,Tanzania,0.708861,0.760573,767,770,0.584094,0.936364,0.901408,0.693269
0,cop,Tanzania,0.742681,0.782694,767,770,0.628422,0.936364,0.907721,0.716700
0,esa,Tanzania,0.741627,0.789200,767,770,0.606258,0.971429,0.954825,0.712381
0,glad,Tanzania,0.718494,0.776187,767,770,0.572360,0.979221,0.964835,0.696858
0,harvest,Togo,0.747541,0.781250,152,200,0.750000,0.805000,0.745098,0.809045
0,cop,Togo,0.569170,0.690341,152,200,0.473684,0.855000,0.712871,0.681275
